In [1]:
import cv2
import numpy as np
import os
from tensorflow import keras
import time
import mediapipe as mp
import csv
import pandas as pd
import seaborn as sns
from matplotlib import patches
import matplotlib.pyplot as plt

from tensorflow import keras
import tensorflow as tf
import time
import datetime
import timeit

from flask import Flask, request, jsonify
from flask_socketio import SocketIO
import json
import requests
import base64

from tensorflow import keras 
from PIL import ImageFont ,ImageDraw ,Image


In [2]:
classes = ['고민', '발가락', '자극', '장애인', '침착', '병문안', '검사', '당뇨병', '면역', '감기',
       '변비', '병명', '보건소', '불면증', '붕대', '설사', '성병', '소화제', '수면제', '회복',
       '입원', '진단서', '치료', '치료법', '퇴원', '한약', '빈혈', '화상', '술', '커피', '의사',
       '간호사', '체온', '금요일', '수요일', '예전', '월요일', '일요일', '화요일', '과로', '실수',
       '정밀검사', '금식', '금연', '금주', '식도염', '환자실', '사용법', '혈액형', '숨차다', '통증',
       '악영향', '촉진', '치료제', '칼', '칼슘', '과음', '마르다', '가렵다', '답답', '건강',
       '불안', '팔', '아프다', '오줌', '혀', '얼굴', '손', '코', '콧물', '춥다', '머리', '몸',
       '턱', '엉덩이', '일어나다', '임신', '싫다', '입', '걷다', '겨드랑이', '부족', '충혈',
       '왼쪽', '오른쪽', '떨다', '귀', '눈썹', '눈', '치아', '이마', '볼', '목', '자다',
       '가족', '전염', '병원', '상처', '병', '밥', '붓다', '피곤', '양치', '중독', '치매',
       '환자', '상담', '충격', '아기', '안색', '몸무게', '간', '노화', '체형', '무기력', '미성년',
       '변화', '위', '인체', '가루약', '감염병', '중병', '물약', '약효', '여드름', '잊어버리다',
       '깜빡하다', '넘기다', '당하다', '온도', '아래', 'stop']

In [3]:
##keypoint 좌표 반화
def extract_keypoints(results):
    pose = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    lh = np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(63)
    rh = np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(63)
    return np.concatenate([pose, lh, rh])

def extract_keypoints_pose(result):
    pose = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    return np.concatenate([pose])

def extract_keypoints_hand(result):
    lh = np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(63)
    rh = np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(63)
    return np.concatenate([lh, rh])   

In [4]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

sequence_lenght = 30

In [5]:
label_map = {label:num for num, label in enumerate(classes)}

In [6]:
label_map

{'고민': 0,
 '발가락': 1,
 '자극': 2,
 '장애인': 3,
 '침착': 4,
 '병문안': 5,
 '검사': 6,
 '당뇨병': 7,
 '면역': 8,
 '감기': 9,
 '변비': 10,
 '병명': 11,
 '보건소': 12,
 '불면증': 13,
 '붕대': 14,
 '설사': 15,
 '성병': 16,
 '소화제': 17,
 '수면제': 18,
 '회복': 19,
 '입원': 20,
 '진단서': 21,
 '치료': 22,
 '치료법': 23,
 '퇴원': 24,
 '한약': 25,
 '빈혈': 26,
 '화상': 27,
 '술': 28,
 '커피': 29,
 '의사': 30,
 '간호사': 31,
 '체온': 32,
 '금요일': 33,
 '수요일': 34,
 '예전': 35,
 '월요일': 36,
 '일요일': 37,
 '화요일': 38,
 '과로': 39,
 '실수': 40,
 '정밀검사': 41,
 '금식': 42,
 '금연': 43,
 '금주': 44,
 '식도염': 45,
 '환자실': 46,
 '사용법': 47,
 '혈액형': 48,
 '숨차다': 49,
 '통증': 50,
 '악영향': 51,
 '촉진': 52,
 '치료제': 53,
 '칼': 54,
 '칼슘': 55,
 '과음': 56,
 '마르다': 57,
 '가렵다': 58,
 '답답': 59,
 '건강': 60,
 '불안': 61,
 '팔': 62,
 '아프다': 63,
 '오줌': 64,
 '혀': 65,
 '얼굴': 66,
 '손': 67,
 '코': 68,
 '콧물': 69,
 '춥다': 70,
 '머리': 71,
 '몸': 72,
 '턱': 73,
 '엉덩이': 74,
 '일어나다': 75,
 '임신': 76,
 '싫다': 77,
 '입': 78,
 '걷다': 79,
 '겨드랑이': 80,
 '부족': 81,
 '충혈': 82,
 '왼쪽': 83,
 '오른쪽': 84,
 '떨다': 85,
 '귀': 86,
 '눈썹': 87,
 '눈': 88,
 '치아': 8

In [7]:
def mediapipe_detection(image,model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image, results

In [8]:
new_model = tf.keras.models.load_model('../detect_body/완성 모델/lstm(90퍼)얼굴추가.h5')
threshold = 0.5

In [12]:
app = Flask(__name__)

global keypoints 
keypoints= []
words = []
@app.route("/image_append", methods=['POST'])
def image_append():
    global keypoints
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        data= request.get_json()
        image = base64.b64decode(data['img'])
        jpg_arr = np.frombuffer(image, dtype=np.uint8)
        image= cv2.imdecode(jpg_arr, cv2.IMREAD_COLOR)
    
        img, results = mediapipe_detection(image, holistic)
        keypoint = extract_keypoints(results)
        keypoint = np.concatenate([keypoint[0:99], keypoint[132:258]])
        
        keypoints.append(keypoint)
        
#         print(keypoints)
        return 'good'
    
@app.route("/predict", methods=['POST'])
def predict():
    global keypoints
    words = []
    for i in range(int(len(keypoints)/30)-1):
        res = new_model.predict(np.expand_dims(keypoints[i*30:(i+1)*30], axis=0))[0]
        if res[np.argmax(res)] > threshold:
                    words.append(classes[np.argmax(res)])
    print(words)
    keypoints = []
    
    return 'good'

    
if __name__ == '__main__': # 
    app.run(host='0.0.0.0')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)


In [ ]:
len(keypoints)

In [86]:
np.shape(keypoints)

(128, 225)

In [87]:
keypoints

[array([ 4.89676744e-01,  2.21234679e-01, -1.79446578e+00,  9.91295099e-01,
         4.20991570e-01,  3.00940871e-01, -1.65618491e+00,  9.87185061e-01,
         3.85996819e-01,  2.95482516e-01, -1.65706527e+00,  9.86526191e-01,
         3.53619069e-01,  2.88763642e-01, -1.65718269e+00,  9.85883772e-01,
         5.06645739e-01,  3.10826421e-01, -1.64604318e+00,  9.82641518e-01,
         5.32488048e-01,  3.15983891e-01, -1.64516330e+00,  9.83373404e-01,
         5.53423643e-01,  3.21758866e-01, -1.64651358e+00,  9.85830843e-01,
         2.92611420e-01,  2.43975416e-01, -8.53928745e-01,  9.88322794e-01,
         5.55812001e-01,  2.87673235e-01, -7.57201672e-01,  9.90934908e-01,
         4.30303514e-01,  1.40621543e-01, -1.50720751e+00,  9.91588771e-01,
         5.42709649e-01,  1.56258821e-01, -1.48808396e+00,  9.92567658e-01,
         1.31007582e-01, -9.62609053e-02, -5.67170739e-01,  8.79074216e-01,
         7.92422056e-01, -4.76664305e-02, -5.94649613e-01,  9.64837968e-01,
        -2.5

In [91]:
for i in range(int(len(keypoints)/30)-1):
        print(len(keypoints[i:(i+1)*30]))
        res = new_model.predict(np.expand_dims(keypoints[i*30:(i+1)*30], axis=0))[0]
        if res[np.argmax(res)] > threshold:
                    words.append(classes[np.argmax(res)])

30
59


ValueError: in user code:

    C:\Users\ace35\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1569 predict_function  *
        return step_function(self, iterator)
    C:\Users\ace35\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1559 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\ace35\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\ace35\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\ace35\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3608 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\ace35\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1552 run_step 